In [1]:
import torch
import torch.nn.functional as F
from datasets import Dataset, concatenate_datasets, load_from_disk
import os
import json
from datasets import (
    Dataset,
    DatasetDict,
    Features,
    Value
)
import pandas as pd

In [7]:
rank = torch.load('/opt/ml/input/code/colbert/rank/test_rank.pth')
score = torch.load('/opt/ml/input/code/colbert/score/score_test.pth')
org_rank = torch.load('/opt/ml/input/code/colbert/inferecne_colbert_rank.pth')
score,_ = score.sort(descending=True)
score = F.softmax(score)
test_dataset = load_from_disk('/opt/ml/input/data/test_dataset')
query = list(test_dataset['validation']["question"])

with open(os.path.join("/opt/ml/input/data/wikipedia_documents.json"), "r", encoding="utf-8") as f:
    wiki = json.load(f)

el_ser = pd.read_csv('/opt/ml/input/code/elastic_retrieval.csv')
el_ser['indices'] = el_ser['indices'].map(eval)
spa_rank = torch.tensor(list(el_ser['indices'].values))
el_ser['scores'] = el_ser['scores'].map(eval)
el_ser['scores'] = spa_score = F.softmax(torch.tensor(list(el_ser['scores'].values))) * 0.7

contexts = list(
    dict.fromkeys([v["text"] for v in wiki.values()])
)  # set 은 매번 순서가 바뀌므로

/tmp/ipykernel_3934/1472470266.py:5: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  score = F.softmax(score)
/tmp/ipykernel_3934/1472470266.py:16: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  el_ser['scores'] = spa_score = F.softmax(torch.tensor(list(el_ser['scores'].values))) * 0.7


In [107]:
rank = rank[:,:20]
score = score[:,:20]

In [108]:
all_rank = torch.cat((rank, spa_rank),dim=1)
all_score = torch.cat((score, spa_score),dim=1)

all_score, indices = torch.sort(all_score, descending=True)

tts = []
for i in range(len(indices)):
    tt = []
    for j in indices[i]:
        tt.append(all_rank[i][j])
    tts.append(tt)

all_rank = torch.tensor(tts)

# sparse, dense를 합쳐서 sorting
# 때문에 중복된 index가 있을 수 있음.

In [109]:
df = test_dataset['validation'].to_pandas()

In [97]:
# tt = []
# context = []
# scores = []
# for i,j in zip(all_rank, all_score):
#     for rnk in i[:20]:
#         context.append(contexts[rnk]) 
#     scores.append(j[:20].tolist())
#     tt.append(context)
#     context = []

# df['context'] = tt
# df['retrieval_score'] = scores

# df1 = df.explode('context')
# df1['retrieval_score'] = df.explode('retrieval_score')['retrieval_score']

# df1['temp'] = [':'+str(i) for j in range(0,600) for i in range(0,20)]
# df1['id'] = df1['id'] + df1['temp']

# sigle passage

In [110]:
# # multi passage
# tt = []
# context = ''
# scores = []
# for i,j in zip(all_rank, all_score):
#     for rnk in i[:20]:
#         context+=contexts[rnk]
#     scores.append(j[:20].tolist())
#     tt.append(context)
#     context = ''

# df['context'] = tt

# df1 = df.copy()

In [8]:
rank

tensor([[20220, 33816, 21600,  ..., 35930, 36882, 52681],
        [ 9787, 23871, 22187,  ..., 27047,  3047, 27054],
        [16750, 16749, 12230,  ...,  2723, 20579,  4294],
        ...,
        [21892, 11780,  8807,  ...,  4291,  4294, 20579],
        [39532, 39518, 39533,  ..., 50379,  8392, 30605],
        [11438, 50859, 44637,  ..., 26974, 27511, 49734]])

In [ ]:
# multi passage and not rerank
tt = []
context = ''
scores = []
for i in rank:
    for rnk in i[:20]:
        context += contexts[rnk]
    tt.append(context)
    context = ''

df['context'] = tt

df1 = df.copy()

In [115]:
f = Features(
    {
        "context": Value(dtype="string", id=None),
        "id": Value(dtype="string", id=None),
        "question": Value(dtype="string", id=None),
    }
)

datasets = DatasetDict({"validation": Dataset.from_pandas(df1, features=f)})

In [116]:
datasets.save_to_disk("/opt/ml/input/data/colbert_result")

In [2]:
datasets = load_from_disk("/opt/ml/input/data/colbert_result")
datasets

DatasetDict({
    validation: Dataset({
        features: ['context', 'id', 'question'],
        num_rows: 600
    })
})

## Retrieval Score를 반영한 새로운 Prediction 만들기

In [3]:
import json

with open('/opt/ml/input/code/outputs/test_dataset/nbest_predictions.json', 'r') as f:
    pred = json.load(f)

In [79]:
mu = 0.7

for key in pred.keys():
    item = pred[key]
    temp_score = df1[df1['id'] == key]['retrieval_score'].values[0]
    for i in range(len(item)):
        prob = pred[key][i]['probability']
        pred[key][i]['probability'] = mu*prob + (1-mu)*temp_score

In [80]:
pred['mrc-0-001606:1']

[{'start_logit': 5.873795032501221,
  'end_logit': 2.805821657180786,
  'text': '석조 궁륭으로 올리는 것',
  'probability': 0.3561535030603409},
 {'start_logit': 5.873795032501221,
  'end_logit': 1.9835196733474731,
  'text': '석조 궁륭',
  'probability': 0.2825936436653137},
 {'start_logit': 5.873795032501221,
  'end_logit': 1.8453035354614258,
  'text': '석조 궁륭으로 올리',
  'probability': 0.27515023648738857},
 {'start_logit': 4.741925239562988,
  'end_logit': 2.805821657180786,
  'text': '천장을 석조 궁륭으로 올리는 것',
  'probability': 0.2672422528266907},
 {'start_logit': 4.741925239562988,
  'end_logit': 1.9835196733474731,
  'text': '천장을 석조 궁륭',
  'probability': 0.2435243759304285}]

모든 passage 가 합쳐져있는 dict만들고 정렬하기!

In [81]:
from collections import defaultdict

new_pred = defaultdict(list)
for key in pred.keys():
    item = pred[key]
    for i in range(len(item)):
        temp_key = key.split(':')[0]
        new_pred[temp_key].append(pred[key][i])
    new_pred[temp_key] = sorted(new_pred[temp_key], key=lambda x: x['probability'], reverse=True)

In [83]:
{key:value[0]['text'] for key, value in new_pred.items()}

{'mrc-1-000653': '지구',
 'mrc-1-001113': '냉전',
 'mrc-0-002191': '대통령인 빌헬름 미클라스',
 'mrc-0-003951': '뉴질랜드',
 'mrc-1-001272': '프랑스',
 'mrc-1-000993': '유두',
 'mrc-0-005021': '계급제',
 'mrc-1-000163': '하코네 산의 화산 폭발',
 'mrc-0-001283': '순조 11년(1811)',
 'mrc-0-004543': '고전도성 철',
 'mrc-0-000439': '점쟁이',
 'mrc-0-002895': 'empty',
 'mrc-0-000535': '롭 포드',
 'mrc-1-001724': '노스햄프턴 교회',
 'mrc-0-000901': '베르나부초',
 'mrc-0-001606': '라틴 십자 플랜',
 'mrc-0-000266': '국민',
 'mrc-0-001326': '‘자유’지에',
 'mrc-0-000032': '국방부 보고서',
 'mrc-0-005215': '입석대(立石臺)',
 'mrc-0-005407': '숙의 정씨',
 'mrc-0-003683': '국가인권위원회',
 'mrc-0-003644': '이탈리아',
 'mrc-0-002835': '망자',
 'mrc-0-000049': '줄거리나 캐릭터',
 'mrc-1-001829': 'empty',
 'mrc-1-001662': '보신 전쟁',
 'mrc-0-001206': '2002년',
 'mrc-0-004007': '신시내티대학교',
 'mrc-1-000418': '일등바위',
 'mrc-0-003133': 'empty',
 'mrc-0-004646': '한신・아와지 대지진',
 'mrc-0-001058': '기원정사',
 'mrc-0-002361': 'empty',
 'mrc-0-004830': '히포캠프',
 'mrc-0-002762': '경복사(景福寺)',
 'mrc-0-000395': '도우카즈토시',
 'mrc-0-001668

### pred확인

In [6]:
pred

{'mrc-1-000653': [{'start_logit': 7.774663925170898,
   'end_logit': 8.994311332702637,
   'text': '지구',
   'probability': 0.9999527931213379},
  {'start_logit': 7.774663925170898,
   'end_logit': -1.3678220510482788,
   'text': '지구에서, 사락사라',
   'probability': 3.16054756694939e-05},
  {'start_logit': -4.172115325927734,
   'end_logit': 8.994311332702637,
   'text': '전 지구',
   'probability': 6.4797700360941235e-06},
  {'start_logit': -4.270423889160156,
   'end_logit': 8.994311332702637,
   'text': '40억년전 지구',
   'probability': 5.873063855688088e-06},
  {'start_logit': 7.774663925170898,
   'end_logit': -3.601935863494873,
   'text': '지구에서, 사락사라의 사람들이',
   'probability': 3.3845351481431862e-06}],
 'mrc-1-001113': [{'start_logit': 9.110544204711914,
   'end_logit': 8.869184494018555,
   'text': '냉전',
   'probability': 0.607577919960022},
  {'start_logit': 9.744996070861816,
   'end_logit': 7.775757789611816,
   'text': '냉전',
   'probability': 0.3839474022388458},
  {'start_logit': 9.7449

In [88]:
id_val = 'mrc-0-001606'

In [105]:
df1[df1['id'].str.contains(id_val)]

,question,id,context,retrieval_score,temp
15,슈파이어 대성당 2차 축조시에 가장 우선순위로 고려했던 천장 건축 방식은?,mrc-0-001606:0,"슈파이어 대성당(Speyerer Dom, 정식 명칭: Domkirche St. Ma...",0.923551,:0
15,슈파이어 대성당 2차 축조시에 가장 우선순위로 고려했던 천장 건축 방식은?,mrc-0-001606:1,2차 건축은 하인리히 4세가 통치하던 1082년에 시작되었다. 2차 건축 작업은 하...,0.321333,:1
15,슈파이어 대성당 2차 축조시에 가장 우선순위로 고려했던 천장 건축 방식은?,mrc-0-001606:2,독일 남서부 라인라트팔츠 주 슈파이어에 위치한 슈파이어 대성당은 1024년부터 11...,0.180599,:2
15,슈파이어 대성당 2차 축조시에 가장 우선순위로 고려했던 천장 건축 방식은?,mrc-0-001606:3,슈파이어 대성당은 11세기~12세기 로마네스크 건축의 발달과 18세기부터 현재까지의...,0.125732,:3
15,슈파이어 대성당 2차 축조시에 가장 우선순위로 고려했던 천장 건축 방식은?,mrc-0-001606:4,"프란츠 이그나츠 미하엘 노이만, 즉 1772년 바로크 시기의 건축가였던 발타자어 노...",0.115367,:4
15,슈파이어 대성당 2차 축조시에 가장 우선순위로 고려했던 천장 건축 방식은?,mrc-0-001606:5,슈파이어 대성당의 복원 과정에서 18세기에 건축가 I. M. 노이만(I. M. Ne...,0.0949924,:5
15,슈파이어 대성당 2차 축조시에 가장 우선순위로 고려했던 천장 건축 방식은?,mrc-0-001606:6,독일의 성기 로마네스크는 통상적으로 1060~1170년 사이를 일컫는데 정치사에서는...,0.0661174,:6
15,슈파이어 대성당 2차 축조시에 가장 우선순위로 고려했던 천장 건축 방식은?,mrc-0-001606:7,독일의 성기 로마네스크는 통상적으로 1060~1170년 사이를 일컫는데 정치사에서는...,0.058384,:7
15,슈파이어 대성당 2차 축조시에 가장 우선순위로 고려했던 천장 건축 방식은?,mrc-0-001606:8,슈파이어 대성당은 황제의 위대함과 힘을 보여주는 대표적인 건축물 이라고 볼 수 있다...,0.0437365,:8
15,슈파이어 대성당 2차 축조시에 가장 우선순위로 고려했던 천장 건축 방식은?,mrc-0-001606:9,"슈파이어 대성당(Speyerer Dom, 정식 명칭: Domkirche St. Ma...",0.0254888,:9


In [91]:
df1[df1['id'].str.contains(id_val)]['question'].values[0]

'슈파이어 대성당 2차 축조시에 가장 우선순위로 고려했던 천장 건축 방식은?'

In [92]:
df1[df1['id'].str.contains(id_val)]['context'].values[0]

'슈파이어 대성당(Speyerer Dom, 정식 명칭: Domkirche St. Maria un St. Stephan)은 독일 남서부 라인란트팔츠주 슈파이어에 있는 대성당이다. 슈파이어의 가톨릭 주교좌가 이곳에 있다. 제국시절의 황제들이 안장되어 있는 곳이라 현지에서는 슈파이어 카이저돔 (Kaiserdom zu Speyer) 또는 슈파이어 돔 (Speyer Dom)으로 불린다. 독일 로마네스크 양식의 시작을 알리는 이 대성당은 폭 30m, 길이 133m에 이르며, 높이가 72m나 되는 탑이 있다. 이 건축물은 로마네스크 양식의 종교 건축물로서는 최대 규모를 자랑한다. 그리고 익랑과 내진 아래에 있는 지하 제실은 천장 높이만 해도 7m에 달해, 지하 제실 중에서 가장 큰 규모를 자랑한다.\n\n슈파이어 대성당은 1030년 황제 콘라트 2세의 명령으로 기공되어 1060년 하인리히 4세에 의해 헌당된 라틴 십자 플랜의 독일 로마네스크 건축을 대표하는 바실리카식 성당이다. 1981년 유네스코에 의해 세계문화유산으로 지정되었다. 1912년, 예술사가인 빌헬름 핀더는 슈파이어 대성당을 보고 다음과 같이 말했다. “당당한 본진은 전함 같은 위용을 보이면서 화려한 버팀 기둥의 윗면을 라인 강 쪽으로 돌리기 위해 서쪽에서 동쪽으로 뻗어 있는 것처럼 보인다.” 성당 건물은 당시 독일 제국(신성 로마 제국)의 위용을 잘 표현하고 있다고 말한다'

In [344]:
el_ser = pd.read_csv('/opt/ml/input/code/elastic_retrieval.csv')
el_ser.loc[16:]

,indices,scores
16,"[4879, 1738, 43521, 8902, 17606, 24074, 47563,...","[14.366098, 14.2824955, 10.417053, 10.063488, ..."
17,"[5810, 38162, 34741, 437, 438, 27217, 36602, 4...","[22.62582, 18.612743, 17.310268, 16.05808, 15...."
18,"[2360, 52367, 31577, 1070, 15135, 28256, 39099...","[19.551682, 18.64961, 18.635862, 18.358938, 18..."
19,"[4723, 912, 46607, 50485, 14947, 51903, 37926,...","[30.737438, 29.792929, 17.735756, 16.878187, 1..."
20,"[5006, 2429, 40854, 40876, 40855, 40859, 40862...","[27.282516, 26.517876, 17.117645, 16.973019, 1..."
...,...,...
595,"[22280, 22277, 22279, 44110, 22276, 56260, 282...","[27.183924, 24.70639, 20.758413, 18.110126, 18..."
596,"[2401, 5000, 2390, 4995, 2380, 2378, 2366, 444...","[36.00628, 34.95566, 27.109509, 26.866253, 21...."
597,"[21892, 45275, 41235, 24308, 31325, 30723, 534...","[23.47238, 22.803211, 22.64158, 22.071846, 22...."
598,"[39532, 39535, 52875, 52693, 39418, 52874, 397...","[35.92101, 30.112059, 25.52641, 24.838594, 24...."
